# In this notebook i will make final fucntion which takes user query and gives top 10 most similar results.

## we have tried different embedding techniques in preprocessing_sc1.ipynb and we have found that Universal sentence encoder gives most similar results so that we will use universal sentence encoding model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import joblib
import numpy as np
import re
import time
from sklearn.metrics.pairwise import cosine_similarity
import os
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/StackOverflowData/pretrain_data_all/featurized_data.csv')
df.shape

(1088418, 10)

In [ ]:
df.head()

,id,preprocessed_title,preprocessed_question,preprocessed_answer,score,tags,question_content,all_text,question_length,title
0,927358,undo recent local commits git,accidentally committed wrong files git push co...,undo commit amp redo command responsible undo ...,24809,git|version-control|git-commit|undo,undo recent local commits git accidentally com...,undo recent local commits git accidentally com...,18,How do I undo the most recent local commits in...
1,2003505,delete git branch locally remotely,failed attempts delete remote branch properly ...,executive summary note cases delete local bran...,19556,git|version-control|git-branch|git-push|git-re...,delete git branch locally remotely failed atte...,delete git branch locally remotely failed atte...,15,How do I delete a Git branch locally and remot...
2,292357,difference git pull git fetch,differences,simplest terms followed updates remote trackin...,13368,git|version-control|git-pull|git-fetch,difference git pull git fetch differences,difference git pull git fetch differences simp...,6,What is the difference between 'git pull' and ...
3,231767,yield keyword,use keyword python example trying understand c...,understand must understand generators understa...,12259,python|iterator|generator,yield keyword use keyword python example tryin...,yield keyword use keyword python example tryin...,38,"What does the ""yield"" keyword do?"
4,477816,json content type use,many quot standards quot json content type one...,json text mime media type json text default en...,11267,json|mime-types|content-type,json content type use many quot standards quot...,json content type use many quot standards quot...,26,Which JSON content type do I use?


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test = train_test_split(df,test_size=0.5,random_state=42)

# reset index of train and test
X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

print('Shape of X_train',X_train.shape)
print('Shape of X_test',X_test.shape)

Shape of X_train (544209, 10)
Shape of X_test (544209, 10)


In [ ]:
X_train.head()

,id,preprocessed_title,preprocessed_question,preprocessed_answer,score,tags,question_content,all_text,question_length,title
0,653714,insert results stored procedure temporary table,without defining data works fine trying using ...,use openrowset look also included sp configure...,1765,sql|sql-server|sql-server-2005|tsql|stored-pro...,insert results stored procedure temporary tabl...,insert results stored procedure temporary tabl...,43,Insert results of a stored procedure into a te...
1,34571,test class private methods fields inner classes,use junit test class internal private methods ...,somewhat legacy java application allowed chang...,3103,java|unit-testing|junit|tdd,test class private methods fields inner classe...,test class private methods fields inner classe...,26,How do I test a class that has private methods...
2,6677035,scroll element jquery,element elements like tag amp tag etc user cli...,assuming button id try example got code articl...,2615,javascript|jquery|scroll,scroll element jquery element elements like ta...,scroll element jquery element elements like ta...,41,Scroll to an element with jQuery
3,1213430,fully delete git repository created init,created git repository would like delete entir...,git keeps files directory remove one init find...,1610,git|git-init,fully delete git repository created init creat...,fully delete git repository created init creat...,16,How to fully delete a git repository created w...
4,1181575,determine whether array contains value,need determine value exists array using follow...,jquery utility function returns index returns ...,1397,javascript|arrays|contains,determine whether array contains value need de...,determine whether array contains value need de...,21,Determine whether an array contains a value


In [ ]:
joblib.dump(X_train,'/content/drive/MyDrive/StackOverflowData/pretrain_data/X_train.csv')

['/content/drive/MyDrive/StackOverflowData/pretrain_data/X_train.csv']

In [ ]:
joblib.dump(X_train,'/content/drive/MyDrive/StackOverflowData/pretrain_data_all/X_train.pkl')

['/content/drive/MyDrive/StackOverflowData/pretrain_data_all/X_train.pkl']

In [ ]:
# below mentioned functions are helper functions which is use for text preprocessing
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"\n", "", phrase)
    return phrase

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

def remove_stopwords(text):
    '''this function will remove stopwords from text '''
    final_text = ''
    for word in text.split():
        if word not in stopwords:
            final_text += word + ' '
    return final_text

def preprocess_title(text):
    # convert to lower case
    text = text.lower()
    # decontract
    text = decontracted(text)
    # remove all punctuations except a-z and c# and c++
    text = re.sub('[^a-zc#c++]+',' ',text)
    # remove stop words
    text = remove_stopwords(text)
    return text

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)
# load universal sentence encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
def embed_text(sentences):
    ''' This function will convert sentence into vector using universal sentence encoder'''
    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        # create vectors from sentences
        embeddings = session.run(embed(sentences))
        # return  embeddings
        return embeddings

In [ ]:
# load X_train
X_train = joblib.load('/content/drive/MyDrive/StackOverflowData/pretrain_data/X_train.csv')
# load question embedding trained using universal sentence encoder
question_embeddings = joblib.load('/content/drive/MyDrive/StackOverflowData/pretrain_data/universal_embed/universal_question_embedding.pkl')

In [ ]:
# load X_train
X_train = joblib.load('/content/drive/MyDrive/StackOverflowData/pretrain_data/X_train.pkl')
# load question embedding trained using universal sentence encoder
question_embeddings = joblib.load('/content/drive/MyDrive/StackOverflowData/pretrain_data/universal_embed/universal_question_embedding.pkl')

In [ ]:
def final_func1(question,top_n=10):
    ''' This function will find top similar result for given query using universal sentence encoder'''
    start = time.time()
    # preprocess question
    question = preprocess_title(question)
    # initialize  vector for user query
    query_vector = embed_text([question])
    similarities =  cosine_similarity((query_vector).reshape(1, -1), Y=question_embeddings, dense_output=True)
    # sort similarities
    sort = np.argsort(similarities[0])
    # get top similarity indices  in descending order
    similarity_index = np.array(list(reversed(sort)))
    # find top n similarities
    top_similarity_index = similarity_index[:top_n]
    # print top similarity values
    print('Top cosine similarities are ======>',similarities[0][top_similarity_index])
    # get similar questions title
    similar_questions = X_train['title'][top_similarity_index]
    total_time = (time.time() - start)
    print('\t')
    print('Total time ===========> ',total_time)
    print('Top 10 similar questions Using Universal sentence encoder')
    print('='*100)
    return list(similar_questions)

In [ ]:
query = 'How to make dictionary in python'
final_func1(query)

Top cosine similarities are ======> [0.63726175 0.6081903  0.60663766 0.5889329  0.5588284  0.54123473
 0.53672063 0.5295532  0.50545824 0.5034981 ]
	
Total time ===========>  12.071171045303345
Top 10 similar questions Using Universal sentence encoder


['What is the Python 3 equivalent of "python -m SimpleHTTPServer"',
 'How can I remove a key from a Python dictionary?',
 'What does __all__ mean in Python?',
 'Create a dictionary with comprehension',
 'Replacements for switch statement in Python?',
 'How do I sort a list of dictionaries by a value of the dictionary?',
 "Does Python have a string 'contains' substring method?",
 'How do I merge two dictionaries in a single expression?',
 'How do I access environment variables in Python?',
 'How do I write JSON data to a file?']